In [1]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit, execute,IBMQ
from qiskit import Aer
from qiskit.tools.visualization import circuit_drawer
from qiskit.tools.visualization import plot_histogram
from qiskit import register, available_backends, get_backend
from math import pi
from qiskit.wrapper.jupyter import *
import matplotlib.pyplot as plt
import sys
from parseRealization import *
%matplotlib inline
global testDir
testDir="./tests/"
#IBMQ.load_accounts()
global ctg

In [2]:
def flipTheString(s):
    myString=""
    length = len(s)
    for it in range(0,length):
        myString=myString+s[length-1-it]
    return myString

def quasmToMatlab(s):
    count = 0
    lines = s.split('\n')
    for line in lines:
        if line.startswith("x"):
            count = count + 1
            tokens = int(line.split(" ")[1].split("[")[1].split("]")[0])+1
            string = str(tokens)+" "+str(tokens)+" 0 "+str(pi)+" "+str(0)
            print(string)
        if line.startswith("swap"):
            count = count + 1
            tokens = line.split("[")
            index1 = int(tokens[1].split("]")[0])+1
            index2 = int(tokens[2].split("]")[0])+1
            string = str(index1)+" "+str(index2)+" 2 "+str(-10)+" "+str(0)
            print(string)
        if line.startswith("cx"):
            count = count + 1
            tokens = line.split("[")
            index1 = int(tokens[1].split("]")[0])+1
            index2 = int(tokens[2].split("]")[0])+1
            q = -15
            if index1>index2:
                q = -20
                t = index1
                index1 = index2
                index2 = t
            string = str(index1)+" "+str(index2)+" 2 "+str(q)+" "+str(0)
            print(string)
        if line.startswith("ry"):
            count = count + 1
            tokens = line.split("(")
            parameter = float(tokens[1].split(")")[0])
            index = int(tokens[1].split("[")[1].split("]")[0])+1
            string = str(index)+" "+str(index)+" 1 "+str(parameter)+" "+str(0)
            print(string)
        if line.startswith("rzz"):
            count = count + 1
            tokens = line.split("(")
            parameter = float(tokens[1].split(")")[0])
            tokenz = tokens[1].split("[")
            index1 = int(tokenz[1].split("]")[0])+1
            index2 = int(tokenz[2].split("]")[0])+1
            
            string = str(index1)+" "+str(index2)+" 2 "+str(parameter)+" "+str(0)
            print(string)
        if line.startswith("rz("):
            count = count + 1
            tokens = line.split("(")
            parameter = float(tokens[1].split(")")[0])
            index = int(tokens[1].split("[")[1].split("]")[0])+1
            string = str(index)+" "+str(index)+" 2 "+str(parameter)+" "+str(0)
            print(string)
        if line.startswith("rx"):
            count = count + 1
            tokens = line.split("(")
            parameter = float(tokens[1].split(")")[0])
            index = int(tokens[1].split("[")[1].split("]")[0])+1
            string = str(index)+" "+str(index)+" 0 "+str(parameter)+" "+str(0)
            print(string)
    print(count)
    
def testFromFile(fileName,details=0):
    fileName1 = testDir+fileName + ".real"
    fileName2 = testDir+fileName + ".pla"
    size = readSizeFromFile(fileName1)
    answers = readKmapFromFile(fileName2)
    errors=[]
    ctg = CircuitTransitionGraph()
    ctg = readGatesFromFile(fileName1,ctg)
    error_count = 0
    backend_sim = Aer.get_backend('qasm_simulator')
    for i in answers:
        qr = QuantumRegister(size)
        cr = ClassicalRegister(size)
        qc = QuantumCircuit(qr,cr)

        for j in range(0,size):
            if i[j]=="1":
                qc.x(qr[j])

        qc,qr = readGatesFromCtg(qc,qr,ctg)
        qc.measure(qr,cr)
        job_sim = execute(qc, backend_sim)#,config)
        stats_sim = job_sim.result().get_counts()
        if details==1:
            # circuit_drawer(qc) won't work, why though
            print(i,stats_sim)
        for release in stats_sim:
            myString = flipTheString(release)
            if myString != answers[i]:
                error = i + ":" + myString + ":"+answers[i]
                errors = errors + [error]
                error_count = error_count+1
    if error_count != 0 :
        print("FAILURE")
    if details==1:
        #print(qc.qasm())
        #quasmToMatlab(qc.qasm())
        print("Input:Received:Expected")
        for item in errors:
            print(item)
        print(answers)
        print("ERROR:",error_count)
    return error_count
            #plot_histogram(stats_sim)

In [3]:
#test not

testNames=["test00","ham3_28","miller","hwb4_52","testVCVC","3_17","test01","simpleSwap","swap1-2","swap1-3", "swap3-2","test1","testCVCV","simpleToffoli","hwb4_52_mod","toffoli"]
fails = 0

#testNames=["0410184"]
for fileName in testNames:
    fails = fails + testFromFile(fileName)
if fails == 0:
    print("SUCCESS")
print(fails,len(testNames))
    

NameError: name 'readKmapFromFile' is not defined